# INTRODUCTION
This notebook performs linear regression on the pre-processed data from "1. daily_import_merge_engineer.ipynb". 

## Libraries

In [58]:
import os
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error   
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

DAILY_DATA_PATH = "data.v3/daily" 

## Import data & column groups

In [2]:
df = pd.read_parquet(os.path.join(DAILY_DATA_PATH, "daily_flights_and_weather_merged.parquet"))

# Flights column groups
flights_terminal_cols = ['flights_arr_A', 'flights_arr_B', 'flights_arr_C', 'flights_arr_D', 'flights_arr_E',
                         'flights_dep_A', 'flights_dep_B', 'flights_dep_C', 'flights_dep_D', 'flights_dep_E']

flights_non_terminal_cols = ['flights_total', 'flights_cancel', 'flights_delay', 'flights_ontime',
                             'flights_arr_ontime', 'flights_arr_delay', 'flights_arr_cancel',
                             'flights_dep_ontime', 'flights_dep_delay', 'flights_dep_cancel']

flights_percentage_cols = ['flights_cancel_pct', 'flights_delay_pct', 'flights_ontime_pct',
                            'flights_arr_delay_pct', 'flights_arr_ontime_pct', 'flights_arr_cancel_pct',
                            'flights_dep_delay_pct', 'flights_dep_ontime_pct', 'flights_dep_cancel_pct']

# Date column groups
date_cols = ['date', 'covid', 'ordinal_date', 'year', 'month', 'day_of_month', 'day_of_week', 'season', 'holiday', 'halloween', 'xmas_eve', 'new_years_eve', 'jan_2', 'jan_3', 'day_before_easter', 'days_until_xmas', 'days_until_thanksgiving', 'days_until_july_4th', 'days_until_labor_day', 'days_until_memorial_day']

# Weather column groups
weather_cols = ['wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction']

# Lag column groups
lag_cols =  ['flights_total_lag_1', 'flights_total_lag_2', 'flights_total_lag_3', 'flights_total_lag_4', 'flights_total_lag_5', 'flights_total_lag_6', 'flights_total_lag_7', 'flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3', 'flights_cancel_lag_4', 'flights_cancel_lag_5', 'flights_cancel_lag_6', 'flights_cancel_lag_7']

# DATA PREPROCESSING

## Train Test Split

In [3]:
# Select training features
train_features = ['random'] + date_cols + weather_cols + lag_cols

# Create X and y
X = df[train_features].drop('date', axis=1)
y = df[flights_non_terminal_cols + flights_percentage_cols]

print(X.columns.tolist())
print("\nTarget columns\n", y.head())

# Split data into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

# Print shapes
print("X_train_full shape:", X_train_full.shape)
print("y_train_full shape:", y_train_full.shape)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_Test shape:", X_test.shape)


['random', 'covid', 'ordinal_date', 'year', 'month', 'day_of_month', 'day_of_week', 'season', 'holiday', 'halloween', 'xmas_eve', 'new_years_eve', 'jan_2', 'jan_3', 'day_before_easter', 'days_until_xmas', 'days_until_thanksgiving', 'days_until_july_4th', 'days_until_labor_day', 'days_until_memorial_day', 'wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction', 'flights_total_lag_1', 'flights_total_lag_2', 'flights_total_lag_3', 'flights_total_lag_4', 'flights_total_lag_5', 'flights_total_lag_6', 'flights_total_lag_7', 'flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3', 'flights_cancel_lag_4', 'flights_cancel_lag_5', 'flights_cancel_lag_6', 'flights_cancel_lag_7']

Target columns
             flights_total  flights_cancel  flights_delay  flights_ontime  \
2018-07-20         1898.0            24.0          430.0          1444.0   
2018-07-21 

## Column transformers

In [4]:
categorical_tranformer = make_pipeline(OneHotEncoder(handle_unknown='ignore')) # Some observed holidays may not be in the training data
numeric_transformer = make_pipeline(StandardScaler())

# print value counts of unique data types in X
print(X.dtypes.value_counts())

# Identify categorical and numeric columns in X_train_full
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include = ['float64', 'float32', 'int32', 'int64']).columns.tolist()

# Check that all columns are accounted for
print(f"categorical columns: {categorical_cols}")
print(f"numeric columns: {numeric_cols}")
print(len(categorical_cols) + len(numeric_cols) == X_train_full.shape[1])

# Linear regression transformer
LR__transformer = ColumnTransformer(
    transformers=[
        ('cat', categorical_tranformer, categorical_cols),
        ('num', numeric_transformer, numeric_cols)
    ])

float64    23
object     11
int64       7
float32     4
int32       2
Name: count, dtype: int64
categorical columns: ['covid', 'month', 'day_of_week', 'season', 'holiday', 'halloween', 'xmas_eve', 'new_years_eve', 'jan_2', 'jan_3', 'day_before_easter']
numeric columns: ['random', 'ordinal_date', 'year', 'day_of_month', 'days_until_xmas', 'days_until_thanksgiving', 'days_until_july_4th', 'days_until_labor_day', 'days_until_memorial_day', 'wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction', 'flights_total_lag_1', 'flights_total_lag_2', 'flights_total_lag_3', 'flights_total_lag_4', 'flights_total_lag_5', 'flights_total_lag_6', 'flights_total_lag_7', 'flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3', 'flights_cancel_lag_4', 'flights_cancel_lag_5', 'flights_cancel_lag_6', 'flights_cancel_lag_7']
True


## Lasso regression

Lasso regression on all targets using gridsearchCV to tune alpha

In [5]:
from sklearn.exceptions import ConvergenceWarning
import warnings

param_grid = {'lasso__alpha': [.01, .1, 1, 10, 20]}

lasso_pipeline = make_pipeline(
    LR__transformer,
    Lasso(max_iter=10000)
)

grid_search = GridSearchCV(
    lasso_pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # n_jobs=-1 means use all available CPU cores
    verbose=0
    )

lasso_models = {}
convergence_issues = {}

# Fit lasso models for all targets
for target in y.columns.tolist():
    grid_search.fit(X_train, y_train[target])

    # Save best model parameters, best alpha, and best model
    best_model = grid_search.best_estimator_
    best_alpha = best_model.named_steps['lasso'].get_params()['alpha']
    lasso_models[f"lasso_{target}"] = grid_search.best_estimator_

    # Identify convergence issues for the best alpha values
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always", ConvergenceWarning)
        best_model.fit(X_train, y_train[target])
        if any(issubclass(warn.category, ConvergenceWarning) for warn in w):
            convergence_issues[target] = best_alpha

# Print convergence issues
if convergence_issues:
    print("Convergence issues:")
    for target, alpha in convergence_issues.items():
        print(f"{target} did not converge with alpha = {alpha}")
else:
    print("No convergence issues for the best alpha values of any target")

/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.349e+04, tolerance: 9.212e+03
  model = cd_fast.enet_coordinate_descent(
/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.088e+04, tolerance: 1.197e+03
  model = cd_fast.enet_coordinate_descent(
/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Obje

No convergence issues for the best alpha values of any target


Best lasso alpha, r-squared, and mean absolute error on validation set for each target

In [28]:
lasso_results = pd.DataFrame(columns=['TARGET', 'ALPHA', 'R2', 'MAE'])

# Print best alpha and R2 for all lasso models
for target in y.columns.tolist():
    model = lasso_models[f"lasso_{target}"]
    alpha = model.named_steps['lasso'].get_params()['alpha']
    r2 = r2_score(y_val[target], model.predict(X_val)).round(3)
    mae = mean_absolute_error(y_val[target], model.predict(X_val)).round(3)
    temp = pd.DataFrame({'TARGET': target, 'ALPHA': alpha, 'R2': r2, 'MAE': mae}, index=[0])
    
    with warnings.catch_warnings():
        warnings.simplefilter(action="ignore", category=FutureWarning)
        lasso_results = pd.concat([lasso_results, temp], ignore_index=True)

print(lasso_results)

                    TARGET  ALPHA     R2      MAE
0            flights_total   0.10  0.916   59.809
1           flights_cancel   1.00  0.823   29.855
2            flights_delay   0.10  0.335   98.026
3           flights_ontime   0.10  0.656  125.792
4       flights_arr_ontime   0.10  0.671   61.727
5        flights_arr_delay   0.10  0.296   50.045
6       flights_arr_cancel   0.10  0.830   14.971
7       flights_dep_ontime   0.10  0.638   64.762
8        flights_dep_delay   0.10  0.391   49.484
9       flights_dep_cancel   0.10  0.797   17.479
10      flights_cancel_pct   0.10  0.805    1.923
11       flights_delay_pct   0.01  0.246    5.483
12      flights_ontime_pct   0.01  0.562    6.338
13   flights_arr_delay_pct   0.10  0.221    5.669
14  flights_arr_ontime_pct   0.10  0.545    6.468
15  flights_arr_cancel_pct   0.10  0.809    1.822
16   flights_dep_delay_pct   0.01  0.283    5.588
17  flights_dep_ontime_pct   0.01  0.586    6.476
18  flights_dep_cancel_pct   0.10  0.787    2.049


## Ridge regression

In [11]:
# Ridge pipeline for flights_ontime
ridge_pipeline = make_pipeline(
    LR__transformer,
    Ridge(alpha=10)
)

# Ridge fit
ridge_pipeline.fit(X_train, y_train['flights_ontime'])

# Ridge predictions
y_pred_ontime = ridge_pipeline.predict(X_val)
print("R2 score:", r2_score(y_val['flights_ontime'], y_pred_ontime))

# Features and coefficients with non-zero coefficients
ridge_ontime_features = ridge_pipeline.named_steps['columntransformer'].get_feature_names_out()
ridge_ontime_coef = ridge_pipeline.named_steps['ridge'].coef_

# Create a dataframe of features and coefficients
ridge_ontime_df = pd.DataFrame({'features': ridge_ontime_features, 'coefficients': ridge_ontime_coef})

# Sort the dataframe by coefficient absolute value, largest to smallest
ridge_ontime_df['coefficients_abs'] = ridge_ontime_df['coefficients'].abs()
ridge_ontime_df.sort_values(by='coefficients_abs', inplace=True, ascending=False)

# Filter the dataframe for coefficients_abs > .1
ridge_ontime_df = ridge_ontime_df[ridge_ontime_df['coefficients_abs'] > .1]

print("Ridge coefficients:\n", ridge_ontime_df)

R2 score: 0.6561574193569667
Ridge coefficients:
                       features  coefficients  coefficients_abs
38   cat__holiday_Thanksgiving   -176.156278        176.156278
82   num__flights_cancel_lag_1    -85.173769         85.173769
70              num__wx_maxref    -82.796434         82.796434
16   cat__day_of_week_Saturday    -66.255361         66.255361
44           cat__xmas_eve_yes    -64.502099         64.502099
..                         ...           ...               ...
25  cat__holiday_Christmas Day      1.750882          1.750882
74      num__wx_wind_direction      1.731846          1.731846
46      cat__new_years_eve_yes      0.767014          0.767014
45       cat__new_years_eve_no     -0.767014          0.767014
68                 num__wx_vis      0.721992          0.721992

[89 rows x 3 columns]


## Ridge regression on all targets using grid search CV to tune alpha

In [12]:
param_grid = {'ridge__alpha': [.01, .1, 1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

ridge_pipeline = make_pipeline(
    LR__transformer,
    Ridge(max_iter=10000)
)

grid_search = GridSearchCV(
    ridge_pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # n_jobs=-1 means use all available CPU cores
    verbose=0
    )

ridge_models = {}
convergence_issues = {}

# Fit lasso models for all targets
for target in y.columns.tolist():
    grid_search.fit(X_train, y_train[target])

    # Save best model parameters, best alpha, and best model
    best_model = grid_search.best_estimator_
    best_alpha = best_model.named_steps['ridge'].get_params()['alpha']
    ridge_models[f"ridge_{target}"] = grid_search.best_estimator_

    # Identify convergence issues for the best alpha values
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always", ConvergenceWarning)
        best_model.fit(X_train, y_train[target])
        if any(issubclass(warn.category, ConvergenceWarning) for warn in w):
            convergence_issues[target] = best_alpha

# Print convergence issues
if convergence_issues:
    print("Convergence issues:")
    for target, alpha in convergence_issues.items():
        print(f"{target} did not converge with alpha = {alpha}")
else:
    print("No convergence issues for the best alpha values of any target")

No convergence issues for the best alpha values of any target


Get alpha, r-squared,and MAE for best ridge fit for each target

In [27]:
ridge_results = pd.DataFrame(columns=['TARGET', 'ALPHA', 'R2', 'MAE'])

# Print best alpha and R2 for all ridge models
for target in y.columns.tolist():
    model = ridge_models[f"ridge_{target}"]
    alpha = model.named_steps['ridge'].get_params()['alpha']
    r2 = r2_score(y_val[target], model.predict(X_val)).round(3)
    mae = mean_absolute_error(y_val[target], model.predict(X_val)).round(3)
    temp = pd.DataFrame({'TARGET': target, 'ALPHA': alpha, 'R2': r2, 'MAE': mae}, index=[0])
    
    with warnings.catch_warnings():
        warnings.simplefilter(action="ignore", category=FutureWarning)
        ridge_results = pd.concat([ridge_results, temp], ignore_index=True)

print(ridge_results)

                    TARGET  ALPHA     R2      MAE
0            flights_total    0.1  0.915   61.295
1           flights_cancel  100.0  0.819   31.497
2            flights_delay   10.0  0.350   96.789
3           flights_ontime    1.0  0.658  125.960
4       flights_arr_ontime    1.0  0.666   62.572
5        flights_arr_delay   10.0  0.304   49.978
6       flights_arr_cancel  100.0  0.834   14.816
7       flights_dep_ontime    1.0  0.639   64.881
8        flights_dep_delay   10.0  0.395   49.687
9       flights_dep_cancel  100.0  0.790   17.057
10      flights_cancel_pct  100.0  0.809    2.036
11       flights_delay_pct   20.0  0.258    5.405
12      flights_ontime_pct   30.0  0.573    6.246
13   flights_arr_delay_pct   30.0  0.233    5.595
14  flights_arr_ontime_pct   30.0  0.540    6.449
15  flights_arr_cancel_pct  100.0  0.818    1.931
16   flights_dep_delay_pct   10.0  0.288    5.576
17  flights_dep_ontime_pct   20.0  0.592    6.411
18  flights_dep_cancel_pct  100.0  0.788    2.187


In [60]:
elastic_net_pipeline = make_pipeline(
    LR__transformer,
    ElasticNet(alpha=10, 
               l1_ratio=0.5,
               max_iter=10000))

# get a list of 200 values from .0001 to .4
alpha_values = [round(x, 4) for x in np.linspace(.0001, .4, 200)]
l1_ratio_values = [round(x, 2) for x in np.linspace(.1, .9, 9)]

param_grid = {'elasticnet__alpha': alpha_values,
              'elasticnet__l1_ratio': l1_ratio_values}

grid_search = GridSearchCV(
    elastic_net_pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # n_jobs=-1 means use all available CPU cores
    verbose=0
    )

elastic_net_models = {}
convergence_issues = {}
models_dir = "models/elastic_net"
os.makedirs(models_dir, exist_ok=True)

for target in y.columns.tolist():
    grid_search.fit(X_train, y_train[target])

    # Save best model parameters, best alpha, and best model
    best_model = grid_search.best_estimator_
    best_alpha = best_model.named_steps['elasticnet'].get_params()['alpha']
    best_l1_ratio = best_model.named_steps['elasticnet'].get_params()['l1_ratio']
    elastic_net_models[f"elastic_net_{target}"] = grid_search.best_estimator_

    # Identify convergence issues for the best alpha values and l1_ratio
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always", ConvergenceWarning)
        best_model.fit(X_train, y_train[target])
        if any(issubclass(warn.category, ConvergenceWarning) for warn in w):
            convergence_issues[target] = (best_alpha, best_l1_ratio)

# Print convergence issues
if convergence_issues:
    print("Convergence issues:")
    for target, alpha_l1_ratio in convergence_issues.items():
        print(f"{target} did not converge with alpha = {alpha_l1_ratio[0]} and l1_ratio = {alpha_l1_ratio[1]}")
else:
    print("No convergence issues for the best alpha and l1_ratio values of any target")

    # print(f"Best parameters for elastic_net_{target}:\n{grid_search.best_params_}")

# Save best elastic net models
for target, model in elastic_net_models.items():
    model_path = os.path.join(models_dir, f"{target}.joblib")
    joblib.dump(model, model_path)
    print(f"Saved {target} model to {model_path}")

/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.235e+06, tolerance: 9.212e+03
  model = cd_fast.enet_coordinate_descent(
/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.662e+06, tolerance: 9.587e+03
  model = cd_fast.enet_coordinate_descent(
/Users/shobbs/.local/share/virtualenvs/predicting_flight_traffic-wZNm__h8/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Obje

No convergence issues for the best alpha and l1_ratio values of any target
Saved elastic_net_flights_total model to models/elastic_net/elastic_net_flights_total.joblib
Saved elastic_net_flights_cancel model to models/elastic_net/elastic_net_flights_cancel.joblib
Saved elastic_net_flights_delay model to models/elastic_net/elastic_net_flights_delay.joblib
Saved elastic_net_flights_ontime model to models/elastic_net/elastic_net_flights_ontime.joblib
Saved elastic_net_flights_arr_ontime model to models/elastic_net/elastic_net_flights_arr_ontime.joblib
Saved elastic_net_flights_arr_delay model to models/elastic_net/elastic_net_flights_arr_delay.joblib
Saved elastic_net_flights_arr_cancel model to models/elastic_net/elastic_net_flights_arr_cancel.joblib
Saved elastic_net_flights_dep_ontime model to models/elastic_net/elastic_net_flights_dep_ontime.joblib
Saved elastic_net_flights_dep_delay model to models/elastic_net/elastic_net_flights_dep_delay.joblib
Saved elastic_net_flights_dep_cancel m

In [61]:
elastic_net_results = pd.DataFrame(columns=['TARGET', 'ALPHA', 'L1 RATIO', 'R2', 'MAE', 'MSE'])

for target in y.columns.tolist():
    model = elastic_net_models[f"elastic_net_{target}"]
    alpha = model.named_steps['elasticnet'].get_params()['alpha']
    l1_ratio = model.named_steps['elasticnet'].get_params()['l1_ratio']
    r2 = r2_score(y_val[target], model.predict(X_val)).round(3)
    mae = mean_absolute_error(y_val[target], model.predict(X_val)).round(3)
    mse = mean_squared_error(y_val[target], model.predict(X_val)).round(1)
    temp = pd.DataFrame({'TARGET': target, 'ALPHA': alpha, 'L1 RATIO': l1_ratio, 'R2': r2, 'MAE': mae, 'MSE': mse}, index=[0])
    
    with warnings.catch_warnings():
        warnings.simplefilter(action="ignore", category=FutureWarning)
        elastic_net_results = pd.concat([elastic_net_results, temp], ignore_index=True)

# Create "model_output" directory
os.makedirs("model_output", exist_ok=True)

# Save results to a csv file
elastic_net_results.to_csv("model_output/elastic_net_results.csv", index=False)

print(elastic_net_results)

                    TARGET   ALPHA  L1 RATIO     R2      MAE      MSE
0            flights_total  0.0021       0.9  0.917   60.724   7868.6
1           flights_cancel  0.3256       0.7  0.818   30.481   4465.6
2            flights_delay  0.0081       0.2  0.350   96.866  23284.6
3           flights_ontime  0.0142       0.9  0.659  125.729  34900.9
4       flights_arr_ontime  0.0142       0.9  0.668   62.248   8839.4
5        flights_arr_delay  0.0142       0.2  0.306   49.727   6170.6
6       flights_arr_cancel  0.2051       0.5  0.835   14.440    999.2
7       flights_dep_ontime  0.0142       0.9  0.639   65.020   9046.9
8        flights_dep_delay  0.0041       0.1  0.393   49.713   5834.8
9       flights_dep_cancel  0.2071       0.5  0.787   16.542   1338.0
10      flights_cancel_pct  0.1367       0.3  0.814    1.927     17.9
11       flights_delay_pct  0.0242       0.9  0.256    5.394     79.9
12      flights_ontime_pct  0.0343       0.4  0.575    6.228    101.5
13   flights_arr_del